In [1]:
import pandas as pd
import numpy as np
import time, os
from scipy import stats

In [2]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)
KERAS_BACKEND="tensorflow"

import tensorflow.keras.backend as backend
import tensorflow.keras.layers as layers

from tensorflow.keras import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

from tensorflow.keras.losses import categorical_crossentropy
from tensorflow_addons.activations import mish

In [3]:
FX = np.load(os.getcwd()+'/pythonGame/saves/FX.npy')
FY = np.load(os.getcwd()+'/pythonGame/saves/FY.npy')

In [4]:
FX.shape,FY.shape

((300, 26), (300, 15))

In [5]:
def custom_loss(y_true, y_pred):    
    tracks = categorical_crossentropy(y_true[:,:10],y_pred[:,:10])
    colors = categorical_crossentropy(y_true[:,10:12],y_pred[:,10:12])
    actions = categorical_crossentropy(y_true[:,12:14],y_pred[:,12:14])
    
    combined = tracks + colors + actions
    return combined * (y_true[:,-1] + 1) # multiplied by pointadv in order to give more weight to better plays

get_custom_objects().update({'custom_loss': custom_loss})

In [6]:
def create_model(summary=True):
    data_in = layers.Input(shape=(26,))
    
    X = layers.Dense(39, activation=mish)(data_in)
    X = layers.Dense(52, activation=mish)(X) 
#     X = layers.Dense(52, activation=mish)(X) # Trying just two layers to start
    
    Tracks = layers.Dense(10, activation='softmax')(X)
    Colors = layers.Dense(2, activation='softmax')(X)
    Action = layers.Dense(2, activation='softmax')(X)
    Pointadv = layers.Dense(1, activation='relu')(X) # dummy var for pointadv, not being trained on for now
    
    Out = layers.Concatenate()([Tracks,Colors,Action,Pointadv])
    
    model = Model(data_in,Out)    
    model.compile(optimizer='adamax', loss=custom_loss)
    
    
    if summary: model.summary()
    return model

In [7]:
model = create_model()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 26)]         0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 39)           1053        input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 52)           2080        dense[0][0]                      
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 10)           530         dense_1[0][0]                    
_______________________________________________________________________________________

In [8]:
np.random.seed(42)
p = np.random.permutation(len(FX))
TX, TY = FX[p[:int(len(FX) * 0.8)]], FY[p[:int(len(FX) * 0.8)]]
VX, VY = FX[p[int(len(FX) * 0.8):]], FY[p[int(len(FX) * 0.8):]]
# splitting data randomly into training and valdiation, arguably should be split by game instead of rows
# more games can be generated later for a better out of sample test.

In [9]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min')

In [10]:
model.fit(TX,TY, epochs=1000, batch_size=8, verbose=1, validation_data=(VX,VY), shuffle=True,
         callbacks=[early_stopping])

Epoch 1/1000
30/30 [==============================] - 0s 6ms/step - loss: 9.8630 - val_loss: 6.1425
Epoch 2/1000
30/30 [==============================] - 0s 2ms/step - loss: 6.5463 - val_loss: 5.2988
Epoch 3/1000
30/30 [==============================] - 0s 2ms/step - loss: 5.7855 - val_loss: 4.7921
Epoch 4/1000
30/30 [==============================] - 0s 2ms/step - loss: 5.3397 - val_loss: 4.4349
Epoch 5/1000
30/30 [==============================] - 0s 2ms/step - loss: 5.0209 - val_loss: 4.1811
Epoch 6/1000
30/30 [==============================] - 0s 2ms/step - loss: 4.7674 - val_loss: 4.0335
Epoch 7/1000
30/30 [==============================] - 0s 2ms/step - loss: 4.5889 - val_loss: 3.8161
Epoch 8/1000
30/30 [==============================] - 0s 2ms/step - loss: 4.3965 - val_loss: 3.6849
Epoch 9/1000
30/30 [==============================] - 0s 2ms/step - loss: 4.2276 - val_loss: 3.6052
Epoch 10/1000
30/30 [==============================] - 0s 2ms/step - loss: 4.1349 - val_loss: 3.4823

In [11]:
model.save(os.getcwd()+'/pythonGame/saves/NeuralNetv1.h5')

In [12]:
# Checking to make sure the model can be loaded with the environment as defined by get_custom_objects.
# I have had one too many models be saved with poorly defined custom objects that refuse to be loaded.
NeuralNet = load_model(os.getcwd()+'/pythonGame/saves/NeuralNetv1.h5', custom_objects=get_custom_objects())

In [13]:
NeuralNet.predict(FX[:60])

array([[4.74179676e-03, 3.16644162e-02, 5.41974247e-01, 7.00855907e-03,
        9.30869021e-03, 1.70686860e-02, 2.29758187e-03, 5.26665058e-03,
        6.10041758e-03, 3.74568909e-01, 6.98311508e-01, 3.01688462e-01,
        9.76247072e-01, 2.37529799e-02, 8.15302968e-01],
       [3.03385872e-03, 7.72135099e-03, 7.34603941e-01, 4.93177539e-03,
        1.70016233e-02, 1.41269602e-02, 2.03517987e-03, 1.23947277e-03,
        1.93828193e-03, 2.13367566e-01, 4.63084728e-02, 9.53691542e-01,
        9.69890833e-01, 3.01092360e-02, 4.79248494e-01],
       [7.71422172e-03, 6.42152131e-02, 4.61704671e-01, 8.76251236e-03,
        1.02448743e-02, 1.78359170e-02, 3.22705647e-03, 1.34199280e-02,
        1.35325361e-02, 3.99343073e-01, 9.05626774e-01, 9.43731889e-02,
        9.97038126e-01, 2.96193594e-03, 1.37677372e+00],
       [5.49030397e-03, 1.47957513e-02, 4.98675883e-01, 8.67867563e-03,
        1.54996710e-02, 3.13119516e-02, 4.10109572e-03, 2.67355167e-03,
        1.47248129e-03, 4.17300612e-0